In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
#import lightgbm as lgb
import time
import pickle
import scipy.stats as st
import multiprocessing

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV,RandomizedSearchCV   #Perforing grid search
from sklearn.preprocessing import LabelBinarizer,LabelEncoder,StandardScaler,MinMaxScaler,robust_scale
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import make_scorer, mean_squared_error 

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [36]:
??sklearn.metrics

In [5]:
#MSE = mean_squared_error(y_true, y_pred)
#RMSE= mean_squared_error(y,y_pred)**0.5 
#RMSE =np.sqrt(MSE)

In [5]:
def rmse(actual_values, predicted_values, convertExp=True):
    """
    - root mean squared log error는 error를 로그화값으로 변환하고, 제곱하고, 평균을 내고, 루트를 씌웁니다.
    - skewness를 해결하기 위해 np.log1p를 했기 때문에, 값을 예측할 때 이를 다시 변환해서 처리해주는 것이 필요합니다. 
    """
    if convertExp==True:
        predicted_values = np.exp(predicted_values),
        actual_values = np.exp(actual_values)
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = np.square(predicted_values - actual_values)
    return np.sqrt(difference.mean())
RMSE = make_scorer(rmse, greater_is_better = False)

In [3]:
def root_mean_squared_error_(ground_truth, predictions):
    return mean_squared_error(ground_truth, predictions) ** 0.5
RMSE = make_scorer(root_mean_squared_error_, greater_is_better=False)  

### XGB regressor

In [5]:
train= pd.read_csv("train_in.csv",encoding="cp949") #nohash+scale0 / #nohash+scale1
test = pd.read_csv("test_in.csv",encoding="cp949")
print(train.shape)
print(test.shape) 

(5801, 14)
(5800, 13)


In [6]:
#train/test 데이터 세팅 + DMatrix화  #나중에 apartment_id도 뺼지 고려.
train_x, train_y = train.drop(["date","number"],axis=1) ,train["number"]
print(train_x.shape)
#test
#data_dmatrix = xgb.DMatrix(data=train_x,label=train_y)
test_s=test.drop("date",axis=1)
print(test_s.shape)

(5801, 12)
(5800, 12)


In [9]:
# 표준화 & 정규화 
# 선형 변환을 적용하여 전체 자료의 분포를 평균 0, 분산 1이 되도록 만드는 과정이다
# 최적화 과정에서의 안정성 및 수렴 속도를 향상
#scaler = MinMaxScaler() #정규화 (때때로 제한)
#scaler = StandardScaler() #표준화 (이상치에 좋음)
#이상치에 좋은 robust_scale
#train_x = scaler.fit_transform(train_x)
#train_x2 = scaler.fit_transform(train_x2)
#test_s =scaler.transform(test_s)
#test_b = scaler.transform(test_b)

In [24]:
#Y값 로그변환 (이미 R에서 함 ) / 서울: log / 부산 log2
#train_y =np.log2(train_y)
#train_y2 =np.log2(train_y2)

### 파라미터 & Search모델 설정

In [3]:
?xgb.XGBRegressor

In [7]:
#gridsearch  params
#https://github.com/albertkklam/XGBRegressor/blob/master/XGBRegressor.ipynb
#1.max_depth/ min_child_weigh
#1-2. 위를 기준으로 +-1, +-1~0.5 미세조정
#2.gamma 
#3.subsample/ colsample_bytree : 0.6~ 1
#3-2. 위를 기준으로 미세조정 0.01단위 
#4.reg_alpha/ reg_lambda
#4-2. *0.2/*0.5/*2/*5
params = {
        "max_depth": [6, 9, 12], #int
        "learning_rate": [0.1, 0.2, 0.3], #float 좀 키워도 됄듯 0.005, 0.05 
        #'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        #'colsample_bylevel': [0.6, 0.7, 0.8, 0.9, 1.0],
        "min_child_weight": [1, 3], #int
        "gamma": [0, 0.5, 1.0, 2.0], #float
        "reg_alpha": [1.0, 5.0],  #float
        "reg_lambda": [1.0, 5.0, 10.0],
        "n_estimators": [150, 200,300] #int 
}
3*3*2*4*2*3*3
#params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
#'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

In [16]:
#randomsearch params 
#rvs
one_to_left = st.beta(10, 1) #베타 연속 확률 변수 
from_zero_positive = st.expon(0, 50) #지수 연속 난수 변수

params = {  
    "n_estimators": st.randint(100, 200), #균일 이산 확률 변수.
    "max_depth": st.randint(3, 10),
    "learning_rate": st.uniform(0.05, 0.4), #균일 연속 확률 변수
    #"colsample_bytree": one_to_left,
    #"subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}
#서울데이터: 1학습: 5~6분/ 부산데이터 1학습: 1~2분
#param_grid = dict(penalty=['l1', 'l2'], dual=[False], tol=[0.0001], C=[1.0], fit_intercept=[True], 
#                  intercept_scaling=[1], class_weight=[None], random_state=[None], solver=['liblinear'], 
#                  max_iter=[100], multi_class=['ovr'], verbose=[0], warm_start=[False], n_jobs=[1])

In [8]:
estimator = xgb.XGBRegressor(#subsample=0.7, 
                             colsample_bytree=0.8,
                             objective= 'reg:linear', 
                             tree_method='hist' , scale_pos_weight=1, seed=1234) #nthread 안쓰임 
                                                              #tree method :auto/exact/approx/hist/gpu_exact/gpu_hist

In [23]:
#CV 설정 
from sklearn.model_selection import KFold
kfold = KFold(n_splits=4,shuffle=True,random_state=1234) #이게 default일듯 
from sklearn.model_selection import RepeatedKFold
rkfold = RepeatedKFold(n_splits=4, n_repeats=2,random_state=1234)
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=3)#,shuffle = False) #계층별 교차검증   타깃값 클래스 비율을 고려
from sklearn.model_selection import RepeatedStratifiedKFold
rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
#?StratifiedKFold #찾아보기 

In [28]:
#model = GridSearchCV(estimator,
#                     param_grid = params, n_jobs=1 , scoring='neg_mean_squared_error',iid=False, cv=kfold,verbose=5) 
#'neg_mean_squared_error'/RMSE /n_jobs /verbose 10 

In [9]:
model = RandomizedSearchCV(estimator,param_distributions = params, 
                           n_jobs=1, n_iter=40,scoring="neg_mean_squared_error",iid=False, cv=4, verbose=True) 
#fit_params={'early_stopping_rounds':20,'eval_set':[(X,y)]},cv=kfold
#'neg_mean_squared_error'/RMSE /n_jobs:1외엔 잘안댕.  /n_iter: 학습하는 랜덤의 조합수  /verbose: n_iter*cv/verbose가 10정도 되게.
#20 cv:3 ->3시간 / 40분
#30 30/cv:3 :5시간 50 /1시간

In [60]:
?RandomizedSearchCV
#RandomizedSearchCV(estimator, param_distributions, n_iter=10, scoring=None, fit_params=None, n_jobs=1, iid=True, 
#refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score='raise')

In [164]:
?GridSearchCV
#GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', 
#error_score='raise')
#https://www.programcreek.com/python/example/104786/sklearn.grid_search.GridSearchCV
#https://wikidocs.net/17858

### 서울 학습모델

In [10]:
%time model.fit(train_x,train_y)

Fitting 4 folds for each of 40 candidates, totalling 160 fits
[08:12:02] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:02] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:02] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:02] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:02] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:03] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:04] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:04] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:05] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:05] Tree method is selected to be 'hist', which u

[08:12:40] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:40] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:41] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:41] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:41] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:42] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:42] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:43] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:44] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:44] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[08:12:45] Tree meth

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:  1.4min finished


[08:13:28] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
Wall time: 1min 27s


RandomizedSearchCV(cv=4, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1234,
       silent=True, subsample=1, tree_method='hist'),
          fit_params={}, iid=False, n_iter=40, n_jobs=1,
          param_distributions={'max_depth': [6, 9, 12], 'learning_rate': [0.1, 0.2, 0.3], 'min_child_weight': [1, 3], 'gamma': [0, 0.5, 1.0, 2.0], 'reg_alpha': [1.0, 5.0], 'reg_lambda': [1.0, 5.0, 10.0], 'n_estimators': [150, 200, 300]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='neg_mean_squared_error', verbose=True)

In [32]:
#저장
with open('xgb_model_s_1213_1.sav', 'wb') as f:
    pickle.dump(model, f)

In [52]:
#불러오기
model = pickle.load(open('xgb_model_s_1213_1.sav', 'rb'))

In [11]:
#print('best_score_:', model.best_score_) #
print('best_params_:', model.best_params_) #
#print('best_estimator_:', model.best_estimator_)

#'reg_lambda': 5.0, 'reg_alpha': 0.1, 'n_estimators': 200, 'min_child_weight': 7, 'max_depth': 15, 
#'learning_rate': 0.1, 'gamma': 5.0, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.8

best_params_: {'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 200, 'min_child_weight': 3, 'max_depth': 12, 'learning_rate': 0.2, 'gamma': 2.0}


In [30]:
#model.grid_scores_

[mean: -876.13641, std: 149.38469, params: {'reg_lambda': 0.1, 'reg_alpha': 5.0, 'n_estimators': 50, 'min_child_weight': 3, 'max_depth': 9, 'learning_rate': 0.4, 'gamma': 0},
 mean: -7580.46646, std: 501.66704, params: {'reg_lambda': 10.0, 'reg_alpha': 50.0, 'n_estimators': 150, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 5.0},
 mean: -864.94331, std: 140.54546, params: {'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 15, 'learning_rate': 0.2, 'gamma': 1.0},
 mean: -882.49723, std: 130.34433, params: {'reg_lambda': 0.1, 'reg_alpha': 5.0, 'n_estimators': 200, 'min_child_weight': 0.5, 'max_depth': 9, 'learning_rate': 0.4, 'gamma': 9.0},
 mean: -748.17671, std: 157.23437, params: {'reg_lambda': 50.0, 'reg_alpha': 1.0, 'n_estimators': 150, 'min_child_weight': 0.5, 'max_depth': 12, 'learning_rate': 0.3, 'gamma': 2.0},
 mean: -755.58795, std: 166.67320, params: {'reg_lambda': 50.0, 'reg_alpha': 1.0, 'n_estimators': 200,

In [26]:
#a=np.log2(1+10)
#np.exp2(a)-1

In [12]:
pred = model.best_estimator_.predict(test_s)
pred =pd.DataFrame(pred)
final = pd.concat([test['date'],pred],axis=1)
final.columns = ['date', 'number']
#best_est = grid.best_estimator_
final = pd.concat([test['date'],pred],axis=1)
#final = final.reset_index()
final.columns = ['date', 'number']
print(final.head())
print(final.shape)

       date       number
0  20180911  1914.193604
1  20180914  1914.193604
2  20180912  1914.193604
3  20180916  1914.193604
4  20180920  1914.193604
(5800, 2)


In [35]:
#final['transaction_real_price'] = np.exp2(final['transaction_real_price'])

In [13]:
final.to_csv("C:/Users/Admin/Desktop/so1.csv", mode='w',index=False)

### 부산 학습모델

In [23]:
%time model.fit(train_x2,train_y2)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 53.2min finished


Wall time: 54min 47s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1234,
       silent=True, subsample=1),
          fit_params={}, iid=False, n_iter=30, n_jobs=1,
          param_distributions={'max_depth': [6, 9, 12, 15, 18], 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5], 'min_child_weight': [0.5, 1, 3, 5, 7, 10], 'gamma': [0, 0.5, 1.0, 2.0, 5.0, 9.0], 'reg_alpha': [0.1, 1.0, 5.0, 10.0, 50.0], 'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0], 'n_estimators': [50, 100, 150, 200]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='neg_mean_squared_error', verbose=True)

In [38]:
#저장
with open('xgb_model_b_12131_1.sav', 'wb') as f:
    pickle.dump(model, f)

In [154]:
#불러오기
model = pickle.load(open('xgb_model_b_1213_1.sav', 'rb'))

In [24]:
#print('best_score_:', model.best_score_) #
print('best_params_:', model.best_params_) #
#print('best_estimator_:', model.best_estimator_)

#'reg_lambda': 0.1, 'reg_alpha': 5.0, 'n_estimators': 150, 'min_child_weight': 0.5, 'max_depth': 9, 
#'learning_rate': 0.1, 'gamma': 1.0, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.9

best_params_: {'reg_lambda': 10.0, 'reg_alpha': 1.0, 'n_estimators': 200, 'min_child_weight': 3, 'max_depth': 15, 'learning_rate': 0.1, 'gamma': 9.0}


In [43]:
#model.grid_scores_

[mean: -99478810.67773, std: 18055914.46490, params: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 400},
 mean: -99846999.16894, std: 18106563.59580, params: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 600},
 mean: -98798320.89296, std: 15546140.79421, params: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 400},
 mean: -99021720.46451, std: 15594170.71449, params: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 600},
 mean: -98730439.36205, std: 17693207.76468, params: {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 400},
 mean: -99108860.99240, std: 18076457.18505, params: {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 600},
 mean: -96565591.09976, std: 17509019.40817, params: {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 8,

In [25]:
#pred = grid.predict(test)
pred = model.best_estimator_.predict(test_b)
pred =pd.DataFrame(pred)
#best_est = grid.best_estimator_
final = pd.concat([test2['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

       key  transaction_real_price
0  1253422             150003216.0
1  1369751             147191680.0
2  1389544             120483928.0
3  1394472             390055200.0
4  1395869             230603584.0
(1281, 2)


In [41]:
#final['transaction_real_price'] = np.exp2(final['transaction_real_price'])

In [26]:
final.to_csv("C:/Users/Admin/Desktop/ZPER 4차/submission/xgb1216_b1.csv", mode='w',index=False)

### scale만 

In [6]:
#scale만 한 데이터
#서울 데이터 
train= pd.read_csv("train1718_s+nohash+scale0.csv") #nohash+scale0 / #nohash+scale1
test = pd.read_csv("test1718_s+nohash+scale0.csv") 
#부산 데이터
train2= pd.read_csv("train1718_b+nohash+scale0.csv")
test2 = pd.read_csv("test1718_b+nohash+scale0.csv")
#train/test 데이터 세팅 + DMatrix화  #나중에 apartment_id도 뺼지 고려.
train_x, train_y = train.drop(["key","Y"],axis=1) ,train["Y"]
train_x2, train_y2 = train2.drop(["key","Y"],axis=1),train2["Y"]
#train.drop(['id', 'target'], inplace=True, axis=1)
#test
#data_dmatrix = xgb.DMatrix(data=train_x,label=train_y)
test_s =test.drop("key",axis=1)
test_b =test2.drop("key",axis=1)
print(test_s.shape)
print(test_b.shape)


(2637, 102)
(1281, 94)


In [7]:
%time model.fit(train_x,train_y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 181.2min finished


Wall time: 3h 4min 57s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1234,
       silent=True, subsample=1),
          fit_params={}, iid=False, n_iter=30, n_jobs=1,
          param_distributions={'max_depth': [6, 9, 12, 15, 18], 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5], 'min_child_weight': [0.5, 1, 3, 5, 7, 10], 'gamma': [0, 0.5, 1.0, 2.0, 5.0, 9.0], 'reg_alpha': [0.1, 1.0, 5.0, 10.0, 50.0], 'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0], 'n_estimators': [50, 100, 150, 200]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='neg_mean_squared_error', verbose=True)

In [12]:
with open('xgb_model_s_1218_scale0.sav', 'wb') as f:
    pickle.dump(model, f)

In [8]:
pred = model.best_estimator_.predict(test_s)
pred =pd.DataFrame(pred)
final = pd.concat([test['key'],pred],axis=1)
final.columns = ['key', 'transaction_real_price']
#best_est = grid.best_estimator_
final = pd.concat([test['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

      key  transaction_real_price
0  462533            1.217938e+09
1  764018            5.816460e+08
2  813528            2.897185e+08
3  845097            4.901704e+08
4  856338            1.852312e+08
(2637, 2)


In [9]:
final.to_csv("C:/Users/Admin/Desktop/ZPER 4차/submission/xgb1216_s2.csv", mode='w',index=False)

In [10]:
%time model.fit(train_x2,train_y2)

NameError: name 'train_x2' is not defined

In [11]:
pred = model.best_estimator_.predict(test_b)
pred =pd.DataFrame(pred)
#best_est = grid.best_estimator_
final = pd.concat([test2['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

ValueError: feature_names mismatch: ['apartment_id', '(Intercept)_1', 'sigungu11140_1', 'sigungu11170_1', 'sigungu11200_1', 'sigungu11215_1', 'sigungu11230_1', 'sigungu11260_1', 'sigungu11290_1', 'sigungu11305_1', 'sigungu11320_1', 'sigungu11350_1', 'sigungu11380_1', 'sigungu11410_1', 'sigungu11440_1', 'sigungu11470_1', 'sigungu11500_1', 'sigungu11530_1', 'sigungu11545_1', 'sigungu11560_1', 'sigungu11590_1', 'sigungu11620_1', 'sigungu11650_1', 'sigungu11680_1', 'sigungu11710_1', 'sigungu11740_1', 'sigungu26110_1', 'sigungu26140_1', 'sigungu26170_1', 'sigungu26200_1', 'sigungu26230_1', 'sigungu26260_1', 'sigungu26290_1', 'sigungu26320_1', 'sigungu26350_1', 'sigungu26380_1', 'sigungu26410_1', 'sigungu26440_1', 'sigungu26470_1', 'sigungu26500_1', 'sigungu26530_1', 'sigungu26710_1', 'sigungu28245_1', '(Intercept)_2', 'transaction_date11~20_2', 'transaction_date21~31_2', '(Intercept)_3', 'heat_typedistrict_3', 'heat_typeindividual_3', 'heat_typeN_3', '(Intercept)_4', 'heat_fuelgas_4', 'heat

In [ ]:
final.to_csv("C:/Users/Admin/Desktop/ZPER 4차/submission/xgb1216_b2.csv", mode='w',index=False)

### scale+log

In [ ]:
#scale+log/sqrt변환한 데이터
#서울 데이터 
train= pd.read_csv("train1718_s+nohash+scale0.csv") #nohash+scale0 / #nohash+scale1
test = pd.read_csv("test1718_s+nohash+scale0.csv") 
#부산 데이터
train2= pd.read_csv("train1718_b+nohash+scale0.csv")
test2 = pd.read_csv("test1718_b+nohash+scale0.csv")
#train/test 데이터 세팅 + DMatrix화  #나중에 apartment_id도 뺼지 고려.
train_x, train_y = train.drop(["key","Y"],axis=1) ,train["Y"]
#train.drop(['id', 'target'], inplace=True, axis=1)
#test
#data_dmatrix = xgb.DMatrix(data=train_x,label=train_y)
test_s =test.drop("key",axis=1)
test_b =test2.drop("key",axis=1)
print(test_s.shape)
print(test_b.shape)

In [ ]:
%time model.fit(train_x,train_y)

In [ ]:
pred = model.best_estimator_.predict(test_s)
pred =pd.DataFrame(pred)
final = pd.concat([test['key'],pred],axis=1)
final.columns = ['key', 'transaction_real_price']
#best_est = grid.best_estimator_
final = pd.concat([test['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

In [ ]:
final['transaction_real_price'] = np.exp(final['transaction_real_price'])

In [ ]:
final.to_csv("C:/Users/Admin/Desktop/ZPER 4차/submission/xgb1216_s3.csv", mode='w',index=False)

In [ ]:
%time model.fit(train_x2,train_y2)

In [ ]:
pred = model.best_estimator_.predict(test_b)
pred =pd.DataFrame(pred)
#best_est = grid.best_estimator_
final = pd.concat([test2['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

In [ ]:
final['transaction_real_price'] = np.exp2(final['transaction_real_price'])

In [ ]:
final.to_csv("C:/Users/Admin/Desktop/ZPER 4차/submission/xgb1216_b3.csv", mode='w',index=False)

### 모델 및 파라미터 설정함수 후 한번에 돌리기

In [ ]:
#n_jobs 활성화 (훨씬 느리다 그냥 쓰지말자. )
def somefunction():
    estimator = xgb.XGBRegressor(subsample=0.7, colsample_bytree=0.7,
                             objective= 'reg:linear', scale_pos_weight=1, seed=1234) 
    params = {
        "max_depth": [6, 8, 10, 12 ,15], #int
        "learning_rate": [0.005, 0.01, 0.05, 0.1, 0.2, 0.3], #float
        #'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        #'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        "min_child_weight": [0.5, 1, 3, 5, 7, 10], #int
        "gamma": [0, 0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 9.0], #float
        "reg_alpha": [0.1, 1.0, 5.0, 10.0, 50.0],  #float
        "reg_lambda": [0.1, 1.0, 5.0, 10.0, 50.0],
        "n_estimators": [100, 125, 150, 175, 200] #int 
    }
    model = RandomizedSearchCV(estimator,param_distributions = params, 
                           n_jobs=1, n_iter=30,scoring="neg_mean_squared_error",iid=False, cv=3, verbose=9) #n_jobs=-1
    %time model.fit(train_x,train_y)
    return model
#
#if __name__ == '__main__':
#    somefunction()
somefunction() #-1 :16~20분.(모델하나당 5~6.5분)
               # 1 :11분

### 시각화

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(estimator,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 10]
plt.show()

In [ ]:
xgb.plot_importance(estimator)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

### ElasticNet (릿지 + 라쏘)

In [27]:
#릿지는 scale에 민감 (규제가 있는 모형은 대부분.)
#alpha: 패널티 항을 곱하는 상수. 기본값은 1.0
from sklearn.linear_model import ElasticNet, Ridge, Lasso
#model = Ridge(alpha=1,solver="cholesky") #"sag"
#라쏘
#model1 =Lasso(alpha=0.1)
#엘라스틱 
#model2 = ElasticNet(alpha=0.1,l1_ratio=0.5) #l1_ratio =0 :릿지 / =1:라쏘
estimator =ElasticNet()

In [160]:
?ElasticNet

In [28]:
params= {'alpha': [1,0.5,0.1,0.05,0.01,0.05,0.001],
         #'alpha': np.arange(1e-4,1e-3,1e-4), #a lower alpha is more likely to overfit (C=1/alpha) np.arange(1e-3,1e-2,1e-3)
         'l1_ratio': np.arange(0.1,1.0,0.1),  #L1(라쏘): 가중치의 절대값이 최소가 되게, 특정 변수들은 모델 생성시 사용 X
         'max_iter':[50,100,200,500,1000]}    #L2(릿지): 라쏘의 경우 가중치가 0이 되지만, 릿지는 가중치가 0에 가깝게 됨.
                 #특성이 많은데 그중 일부분만 중요하다면 라쏘가, 특성의 중요도가 전체적으로 비슷하다면 릿지가 좀 더 괜찮은 모델을 찾아줌 
    
#20 *cv3: 2분
#500   -> 50분?

In [29]:
model = GridSearchCV(estimator,
                     param_grid = params, n_jobs=1 , scoring='neg_mean_squared_error',iid=False, cv=3,verbose=True) #False

In [12]:
#model = RandomizedSearchCV(estimator,
#            param_distributions = params, n_jobs=1, n_iter=20,scoring="neg_mean_squared_error",iid=False, cv=3,verbose=1) 

In [37]:
%time model.fit(train_x,train_y)

In [15]:
with open('elastic_model_s_1211.sav', 'wb') as f:
    pickle.dump(model, f)

In [23]:
#불러오기
model = pickle.load(open('elastic_model_s_1211.sav', 'rb'))

In [31]:
print('best_params_:', model.best_params_) #

best_params_: {'alpha': 0.0001, 'l1_ratio': 0.5, 'max_iter': 200}


In [32]:
pred = model.best_estimator_.predict(test_s)
pred =pd.DataFrame(pred)
final = pd.concat([test['key'],pred],axis=1)
final.columns = ['key', 'transaction_real_price']
#best_est = grid.best_estimator_
final = pd.concat([test['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

      key  transaction_real_price
0  462533            1.415346e+09
1  764018            9.590182e+08
2  813528            4.312191e+08
3  845097            3.060653e+08
4  856338           -6.910453e+07
(2637, 2)


In [33]:
final.to_csv("elastic1211_s.csv", mode='w',index=False)

In [38]:
%time model.fit(train_x2,train_y2)

In [29]:
with open('elastic_model_b_1211.sav', 'wb') as f:
    pickle.dump(model, f)

In [30]:
#불러오기
model = pickle.load(open('elastic_model_b_1211.sav', 'rb'))

In [31]:
print('best_params_:', model.best_params_) #

best_params_: {'max_iter': 200, 'l1_ratio': 0.9, 'alpha': 0.0001}


In [35]:
pred = model.best_estimator_.predict(test_b)
pred =pd.DataFrame(pred)
#best_est = grid.best_estimator_
final = pd.concat([test2['key'],pred],axis=1)
#final = final.reset_index()
final.columns = ['key', 'transaction_real_price']
print(final.head())
print(final.shape)

       key  transaction_real_price
0  1253422            2.274509e+08
1  1369751            1.705920e+08
2  1389544            1.415848e+08
3  1394472            4.588886e+08
4  1395869            2.321004e+08
(1281, 2)


In [36]:
final.to_csv("elastic1211_b.csv", mode='w',index=False)

In [ ]:
#early stopping

### LGB

In [ ]:
train = pd.read_csv('../input/train.csv')

# get the labels
y = train.target.values
train.drop(['id', 'target'], inplace=True, axis=1)

x = train.values

# Create training and validation sets
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# Create the LightGBM data containers
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)


# Train the model
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

# Create a submission
submission = pd.read_csv('../input/test.csv')
ids = submission['id'].values
submission.drop('id', inplace=True, axis=1)


x = submission.values
y = model.predict(x)

output = pd.DataFrame({'id': ids, 'target': y})
output.to_csv("submission.csv", index=False)